In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from glob import glob
import os
import time
import cv2
import pandas as pd
import numpy as np
import random
import zipfile
import sys
import shutil
import sklearn.metrics as metrics
from keras.callbacks import ModelCheckpoint

In [ ]:
import zipfile
# imagens originais
!unzip 'D:\Vitoria\Documentos\galaxyzoo\images\originais.zip'

# imagens segmentadas
# !unzip '/content/drive/MyDrive/projetoGalaxy/imagens/segmentadas.zip'

# ***Funções***

In [ ]:
# importando script das funções
sys.path.append('/content/drive/MyDrive/projetoGalaxy')
from funcoes import *

# ***Mover as pastas (test | train | validation) para fora da pasta (originais ou segmentadas)***

In [ ]:
moverpastas('originais')
# moverpastas('segmentadas')

# ***Pré-Processamento***

In [ ]:
# abrindo dataframe de test
testdf = pd.read_csv("/content/drive/MyDrive/projetoGalaxy/tabelatest.csv", sep=',')

colunas = ['images', 'dr7objid', 'gz2_class', 'asset_id']
testdf = testdf[colunas]
testdf.head(15)

In [ ]:
RotuloImagensTest = list(testdf['gz2_class'].map(lambda nome: int(nome)))
print(RotuloImagensTest)

***Contagem de imagens por pasta e rótulo***

In [ ]:
contagemimagens(RotuloImagensTest)

In [ ]:
directory_train = 'train'
directory_validation = 'validation'
directory_test = 'test'

# ResNet152V2: 224 || InceptionResNetV2: 299 || Inception v3: 299 || Alexnet: 227 || MobileNetV2: 424 || AttentionResNet56: 224 || AttentionResNet92: 224 || GoogLenet: 224 || CoAtNet4: 224
train_ds = train_test_validation_split(directory_train, 227)
val_ds = train_test_validation_split(directory_validation, 227)
test_ds = train_test_validation_split(directory_test, 227)

# ***Modelos***

In [ ]:
# importando script dos modelos
sys.path.append('/content/drive/MyDrive/projetoGalaxy')
from models import *


# importando script do CoAtNet4
sys.path.append('/content/drive/MyDrive/projetoGalaxy/keras_cv_attention_models/attention_layers')
from __init__ import *

***ResNet152V2***

In [ ]:
model = ResNet152V2()

***InceptionResNetV2***

In [ ]:
model = InceptionResNetV2()

***InceptionV3***

In [ ]:
model = InceptionV3()

***MobileNetV2***

In [ ]:
model = MobileNetV2()

***Alexnet***

In [ ]:
model = Alexnet()

***AttentionResNet92***

In [ ]:
# model = AttentionResNet92()

***AttentionResNet56***

In [ ]:
# model = AttentionResNet56()

***GoogLeNet***

In [ ]:
# model = GoogLeNet()

***CoAtNet4***

In [ ]:
# model = CoAtNet4()

# ***Treinamento do Modelo***

In [ ]:
model.summary()

In [ ]:
filepath = '/content/drive/MyDrive/projetoGalaxy/teste'

callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    save_freq="epoch",
    options=None,
    # **kwargs
)

In [ ]:
# define loss, metrics, optimizer
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fits the model on batches with real-time data augmentation
model.fit(train_ds, epochs=20, validation_data=val_ds, validation_freq=1, callbacks=callbacks)

# ***Gerenciar Modelos***

***Carregar Modelo***

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/projetoGalaxy/teste')

# ***Méricas***

In [ ]:
pred = model.predict(test_ds, batch_size = 5, verbose=1)

In [ ]:
pred[:10]

In [ ]:
predicao = np.argmax(pred, axis=-1)
for x in predicao[:10]:
  print(x)

In [ ]:
print(metrics.confusion_matrix(RotuloImagensTest, predicao))

In [ ]:
print(metrics.classification_report(RotuloImagensTest, predicao))

In [ ]:
print(metrics.roc_auc_score(RotuloImagensTest, predicao))

In [ ]:
print(metrics.accuracy_score(RotuloImagensTest, predicao))

In [ ]:
print(metrics.f1_score(RotuloImagensTest, predicao))

In [ ]:
print(metrics.recall_score(RotuloImagensTest, predicao))

# ***NOTES***

*   (X) MobileNetV2
*   (X) ResNet152V2
*   (X) Alexnet
*   (X) InceptionV3
*   (X) InceptionResNetV2
*   (X) AttentionResNet92
*   (X) AttentionResNet56

*   ( ) GoogLeNetX1
*   ( ) GoogLeNetX2
*   ( ) GoogLeNetX3
*   ( ) CoAtNet4
